In [33]:
import torch
from torch.distributions import Uniform as t_unif

In [24]:
init_pop = torch.tensor([[1,1,1,1,1,1,1,1], 
                         [2,2,2,2,2,2,2,2], 
                         [3,3,3,3,3,3,3,3], 
                         [4,4,4,4,4,4,4,4], 
                         [1,2,3,4,1,2,3,4], 
                         [4,3,2,1,4,3,2,1], 
                         [1,2,1,2,1,2,1,2], 
                         [3,4,3,4,3,4,3,4]]) 

In [25]:
def n_satisfied(state):
    n_s = 0
    A, B, C, D, E, F, G, H = (var for var in state)
    
    if (A > G):
        n_s += 1
    if (A <= H):
        n_s += 1
    if (abs(F - B) == 1):
        n_s += 1
    if (G < H):
        n_s += 1
    if (abs(G - C) == 1):
        n_s += 1
    if ((H - C) % 2 == 0):
        n_s += 1
    if (H != D):
        n_s += 1
    if (D >= G):
        n_s += 1
    if (D != C):
        n_s += 1
    if (E != C):
        n_s += 1
    if (E < D - 1):
        n_s += 1
    if (E != H - 2):
        n_s += 1
    if (G != F):
        n_s += 1
    if (H != F):
        n_s += 1
    if (C != F):
        n_s += 1
    if (D != F - 1):
        n_s += 1
    if (abs(E - F) % 2 == 1):
        n_s += 1
        
    return n_s

In [26]:
n_satisfied(init_pop[4])

12

In [27]:
def get_survival_ranges(population, fitness_fn):
    s_arr = []
    
    for i in range(len(population)):
        n_s = fitness_fn(population[i])
        s_arr.append(n_s)
        
    s = sum(s_arr)    
    for i in range(len(s_arr)):
        print(f"State {i + 1} --> Fitness score: {s_arr[i]} Parent likelihood: {torch.math.floor((s_arr[i] / s) * 10000)/100}%")
        s_arr[i] /= s
        if (i != 0):
            s_arr[i] += s_arr[i - 1]        
    
    return s_arr

In [28]:
sr = get_survival_ranges(init_pop, n_satisfied)

State 1 --> Fitness score: 5 Parent likelihood: 8.47%
State 2 --> Fitness score: 5 Parent likelihood: 8.47%
State 3 --> Fitness score: 5 Parent likelihood: 8.47%
State 4 --> Fitness score: 5 Parent likelihood: 8.47%
State 5 --> Fitness score: 12 Parent likelihood: 20.33%
State 6 --> Fitness score: 9 Parent likelihood: 15.25%
State 7 --> Fitness score: 9 Parent likelihood: 15.25%
State 8 --> Fitness score: 9 Parent likelihood: 15.25%


In [34]:
def get_random_parents_crossover(survival_range, num_pairs):
    parent_pairs = []
    
    for i in range(num_pairs):
        p1 = t_unif(0, 1).sample()
        p2 = t_unif(0, 1).sample()

        for j in range(len(survival_range)):
            if (p1 < survival_range[j]):
                p1 = j
                break

        for j in range(len(survival_range)):
            if (p2 < survival_range[j]):
                p2 = j
                break
            
        mutation_var = t_unif(0, len(survival_range) - 1).sample()

        parent_pairs.append([p1, p2, mutation_var])
    
    return parent_pairs

In [35]:
get_random_parents_crossover(sr, 4)

[[6, 0, tensor(3.3052)],
 [2, 6, tensor(2.8595)],
 [3, 6, tensor(2.3276)],
 [1, 4, tensor(4.2796)]]

In [39]:
def genetic(init_pop, domain, fitness_fn, num_iters, max_satisfy):
    pop_ = init_pop
    
    # Lookup table to convert indices to variable names
    int_to_var = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E", 5: "F", 6: "G", 7: "H"}
    
    # Genetic algorithm for-loop
    for i in range(num_iters):
        print(f"Generation {i}:")
        for i in range(len(pop_)):
            print(f"State {i + 1}: {pop_[i]}")
        
        # Getting fitness scores and corresponding survival ranges
        sr = get_survival_ranges(pop_, fitness_fn)
        
        # Getting 4 parent pairs and a random crossover point for each pair (Selection)
        parent_pairs = get_random_parents_crossover(sr, int(len(pop_) / 2))
        
        # Initializing an empty array for the next generation
        new_gen = []
        
        # Performing crossing for each of the parent pairs about the crossover points (Crossing)
        for j in range(len(parent_pairs)):
            parent_pair = parent_pairs[j]
            
            c1 = pop_[parent_pair[0]]
            c2 = pop_[parent_pair[1]]
            crossover_point = int(parent_pair[2].item())
            
            temp = c1[crossover_point: ]
            
            c1[crossover_point: ] = c2[crossover_point: ]
            c2[crossover_point: ] = temp
            
            new_gen.append(c1)
            new_gen.append(c2)
         
        # Performing mutation with a 30% chance of a variable in a state being mutated (Mutation)
        for j in range(len(new_gen)):
            curr_c = new_gen[j]
            
            if (t_unif(0, 1).sample() <= 0.3):
                mutation_var_i = int(t_unif(0, len(c1) - 1).sample().item())
                mutate_to = domain[int(t_unif(0, len(domain) - 1).sample().item())]

                c_i_before = curr_c[mutation_var_i]
                mutated_var = domain[t_unif(0, len(domain) - 1).sample()]
                
                curr_c[mutation_var_i] = mutated_var

                print(f"Child {j + 1}: Variable {int_to_var[mutation_var_i]} mutated to {curr_c[mutation_var_i]} (original value : {c_i_before})")
            else:
                print(f"Child {j + 1}: No mutation")
        
        # Updating population to be the new generation
        pop_ = new_gen
        
        # Checking to see if any state satisfies all constraints
        for i in range(len(pop_)):
            if (fitness_fn(pop_[i]) == max_satisfy):
                print(f"Found solution: {pop_[i]}")
                return pop_

        print("")
    
    return pop_

    

In [40]:
genetic(init_pop, [1, 2, 3, 4], n_satisfied, 3, 17)

Generation 0:
State 1: tensor([1, 1, 1, 1, 1, 1, 1, 1])
State 2: tensor([2, 2, 2, 2, 2, 2, 2, 2])
State 3: tensor([3, 3, 3, 3, 3, 3, 3, 3])
State 4: tensor([4, 4, 4, 4, 4, 4, 4, 4])
State 5: tensor([1, 2, 3, 4, 1, 2, 3, 4])
State 6: tensor([4, 3, 2, 1, 4, 3, 2, 1])
State 7: tensor([1, 2, 1, 2, 1, 2, 1, 2])
State 8: tensor([3, 4, 3, 4, 3, 4, 3, 4])
State 1 --> Fitness score: 5 Parent likelihood: 8.47%
State 2 --> Fitness score: 5 Parent likelihood: 8.47%
State 3 --> Fitness score: 5 Parent likelihood: 8.47%
State 4 --> Fitness score: 5 Parent likelihood: 8.47%
State 5 --> Fitness score: 12 Parent likelihood: 20.33%
State 6 --> Fitness score: 9 Parent likelihood: 15.25%
State 7 --> Fitness score: 9 Parent likelihood: 15.25%
State 8 --> Fitness score: 9 Parent likelihood: 15.25%
Child 1: No mutation
Child 2: No mutation


IndexError: tensors used as indices must be long, int, byte or bool tensors

In [38]:
int(torch.tensor(1).item())

1